In [6]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

import sys
import os

# Get the directory where this notebook is located
notebook_dir = os.path.dirname(os.path.abspath(''))

# Add the PythonPrep directory to the path
pythonprep_dir = os.path.join(os.path.dirname(notebook_dir), 'PythonPrep') if 'PythonPrep' not in notebook_dir else notebook_dir
sys.path.append(pythonprep_dir)

from paths import main_path

In [7]:
### ftti

In [ ]:
path_input = main_path + "/Democracy/Democracy_Main/MainAnalysis/input"
file_path = f"{path_input}/channels21st/protectionism/economic-freedom-of-the-world-2021-master-index-data-for-researchers-iso.xlsx"
df = pd.read_excel(file_path, sheet_name="EFW Data 2021 Report", usecols="B:BV", skiprows=4)
df.columns = df.columns.str.lower()
columns_to_keep = ['iso_code_3', 'year', '4  freedom to trade internationally']
df_final = df[columns_to_keep]
df_final = df_final.rename(columns={
    '4  freedom to trade internationally': 'ftti'
})
df_filtered = df_final[(df_final['year'] > 2000) & (df['year'] <= 2019)]
df_sorted = df_filtered.sort_values(by=['iso_code_3', 'year'], ascending=[True, True])
df_sorted = df_sorted.reset_index(drop=True)

variable = 'ftti'
df_sorted['growth_rate'] = df_sorted.groupby('iso_code_3')[variable].pct_change() * 100

result_df = df_sorted.groupby('iso_code_3').agg(
    **{f'change_{variable}_2001_2019': (variable, lambda x: x.iloc[-1] - x.iloc[0])}, 
    **{f'mean_{variable}_2001_2019': (variable, 'mean')},  
    **{f'gr_{variable}_2001_2019': ('growth_rate', 'mean')}
).reset_index()

df_2000 = df_final[df_final['year'] == 2000]
pivot_df = df_2000.pivot_table(index='iso_code_3', columns='year', values=variable).reset_index()
df_renamed = pivot_df.rename(columns={2000: f'con_{variable}_2000'})
merged_df = pd.merge(result_df, df_renamed, on='iso_code_3', how='left')
result_df = merged_df.dropna()

In [9]:
result_df

,iso_code_3,change_ftti_2001_2019,mean_ftti_2001_2019,gr_ftti_2001_2019,con_ftti_2000
1,ALB,1.720162,7.144310,1.396403,6.321591
2,ARE,-0.901650,8.572336,-0.541646,9.377333
3,ARG,0.044962,5.704836,2.060925,7.403484
5,AUS,-0.081066,7.718634,-0.045790,8.042406
6,AUT,-0.643670,8.459424,-0.377097,9.198202
...,...,...,...,...,...
158,USA,-0.903477,8.159677,-0.594689,8.805980
159,VEN,-3.715524,4.106612,0.826772,7.959691
162,ZAF,-0.143319,6.564427,-0.075357,6.650004
163,ZMB,-0.625742,7.504418,-0.366704,8.124611


In [ ]:
result_df.to_csv(main_path + '/Democracy/Democracy_Main/MainAnalysis/input/channels21st/old_channels/fraser.csv', index=False)

